# World Setup - RagVenture

Dieses Notebook erstellt die Spielwelt für RagVenture mit validiertem Schema.

**Vorgehen:**
1. Schema-Konstanten definieren (Relationship-Types, Properties)
2. Helper-Funktionen für typsichere Erstellung
3. Constraints & Indexes anlegen
4. Spielwelt erstellen (Locations, Items, NPCs, Player)
5. Relationships erstellen (Connections, Placements, Interactions)

In [21]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer, util

load_dotenv(dotenv_path='../.env')


True

In [22]:
driver = GraphDatabase.driver(
    uri=os.getenv('NEO4J_URI'),
    auth=(
        os.getenv('NEO4J_USER'), 
        os.getenv('NEO4J_PASSWORD')
    )
)

print('Docker starten ;-)')
print(f'URI: {os.getenv("NEO4J_URI")}')
print(f'UI:  http://localhost:7474')

# Queryhelper
def run_query(query, params=None):
    
    # Ergebnisliste
    results = []

    # Session öffnen
    with driver.session() as session:

        # Query
        result = session.run(query, params or {})

        # Ergebnisse wegspeichern
        records = [r.data() for r in result]
        results.extend(records)

    return results

Docker starten ;-)
URI: bolt://localhost:7687
UI:  http://localhost:7474


## Schema-Definition

Definiert erlaubte Relationships und Properties um Tippfehler zu vermeiden.

In [23]:
# ============================================================================
# SCHEMA-KONSTANTEN (verhindert Tippfehler!)
# ============================================================================

# Erlaubte Relationship-Types
REL_IST_IN = "IST_IN"               # Item/NPC/Player → Location
REL_TRÄGT = "TRÄGT"                 # Player/NPC → Item (Inventar)
REL_ERREICHT = "ERREICHT"           # Location ↔ Location (bidirektional)
REL_ÖFFNET = "ÖFFNET"               # Item → Item/Location (Schlüssel → Truhe/Tür)
REL_KANN_ANZÜNDEN = "KANN_ANZÜNDEN" # Item → Item (Streichhölzer → Fackel)
REL_BELEUCHTET = "BELEUCHTET"       # Item → Location (Fackel → Finsterwald)
REL_KANN_BRECHEN = "KANN_BRECHEN"   # Item → Item (Hammer → Truhe)

# Standard Item Properties
DEFAULT_ITEM_PROPS = {
    'is_takeable': True,
    'is_usable': False,
    'is_readable': False,
    'is_container': False,
    'is_locked': None,
    'is_lit': None,
    'is_light_source': False,
    'item_type': None,
    'weight': None
}

# Standard Location Properties
DEFAULT_LOCATION_PROPS = {
    'is_dark': False,
    'requires_light': False,
    'is_locked': False
}

print("✓ Schema-Konstanten geladen")
print(f"  Relationships: {[REL_IST_IN, REL_TRÄGT, REL_ERREICHT, REL_ÖFFNET, REL_KANN_ANZÜNDEN, REL_BELEUCHTET, REL_KANN_BRECHEN]}")

✓ Schema-Konstanten geladen
  Relationships: ['IST_IN', 'TRÄGT', 'ERREICHT', 'ÖFFNET', 'KANN_ANZÜNDEN', 'BELEUCHTET', 'KANN_BRECHEN']


## Helper-Funktionen

Typsichere Funktionen zum Erstellen von Nodes und Relationships.

In [24]:
# ============================================================================
# HELPER-FUNKTIONEN
# ============================================================================

def create_item(id, name, description, synonyms=None, **kwargs):
    """
    Erstellt ein Item mit automatischen Embeddings
    
    Args:
        id: Unique identifier (lowercase, no spaces)
        name: Display name
        description: Text description
        synonyms: Liste von Synonym-Strings (optional)
        **kwargs: Zusätzliche Properties (is_usable, is_lit, etc.)
    
    Returns:
        Query result
    """
    # Embeddings generieren
    name_emb = emb_model.encode(name)
    description_emb = emb_model.encode(description)
    synonyms_emb = emb_model.encode(' '.join(synonyms)) if synonyms else None
    
    # Properties zusammenbauen
    props = {
        'id': id.lower(),
        'name': name,
        'description': description,
        'name_emb': name_emb,
        'description_emb': description_emb
    }
    
    if synonyms_emb is not None:
        props['synonyms_emb'] = synonyms_emb
    
    # Defaults + Custom Properties
    for key, default_value in DEFAULT_ITEM_PROPS.items():
        props[key] = kwargs.get(key, default_value)
    
    # Query
    query = """
    CREATE (i:Item $props)
    RETURN i
    """
    
    result = run_query(query, {'props': props})
    print(f"  ✓ Item erstellt: {id}")
    return result


def create_location(id, name, description, **kwargs):
    """
    Erstellt eine Location mit automatischen Embeddings
    
    Args:
        id: Unique identifier
        name: Display name
        description: Text description
        **kwargs: Zusätzliche Properties (is_dark, requires_light, etc.)
    
    Returns:
        Query result
    """
    # Embeddings generieren
    name_emb = emb_model.encode(name)
    description_emb = emb_model.encode(description)
    
    # Properties
    props = {
        'id': id.lower(),
        'name': name,
        'description': description,
        'name_emb': name_emb,
        'description_emb': description_emb
    }
    
    # Defaults + Custom
    for key, default_value in DEFAULT_LOCATION_PROPS.items():
        props[key] = kwargs.get(key, default_value)
    
    query = """
    CREATE (l:Location $props)
    RETURN l
    """
    
    result = run_query(query, {'props': props})
    print(f"  ✓ Location erstellt: {id}")
    return result


def create_npc(id, name, description, **kwargs):
    """
    Erstellt einen NPC mit automatischen Embeddings
    
    Args:
        id: Unique identifier
        name: Display name
        description: Text description
        **kwargs: dialogue, is_trader, is_quest_giver
    
    Returns:
        Query result
    """
    # Embeddings
    name_emb = emb_model.encode(name)
    description_emb = emb_model.encode(description)
    
    props = {
        'id': id.lower(),
        'name': name,
        'description': description,
        'name_emb': name_emb,
        'description_emb': description_emb,
        'dialogue': kwargs.get('dialogue', None),
        'is_trader': kwargs.get('is_trader', False),
        'is_quest_giver': kwargs.get('is_quest_giver', False)
    }
    
    query = """
    CREATE (n:NPC $props)
    RETURN n
    """
    
    result = run_query(query, {'props': props})
    print(f"  ✓ NPC erstellt: {id}")
    return result


def create_player(id='player', name='Player', description='Du bist der Spieler'):
    """
    Erstellt den Player-Node
    
    Returns:
        Query result
    """
    query = """
    CREATE (p:Player {id: $id, name: $name, description: $description})
    RETURN p
    """
    
    result = run_query(query, {'id': id, 'name': name, 'description': description})
    print(f"  ✓ Player erstellt: {id}")
    return result


def create_relationship(from_id, rel_type, to_id):
    """
    Erstellt eine Relationship zwischen zwei Nodes
    
    Args:
        from_id: ID des Start-Nodes
        rel_type: Relationship-Type (nutze REL_* Konstanten!)
        to_id: ID des End-Nodes
    
    Returns:
        Query result
    """
    # Warnung bei unbekannten Relationship-Types
    known_rels = [REL_IST_IN, REL_TRÄGT, REL_ERREICHT, REL_ÖFFNET, 
                  REL_KANN_ANZÜNDEN, REL_BELEUCHTET, REL_KANN_BRECHEN]
    
    if rel_type not in known_rels:
        print(f"  ⚠️  WARNUNG: Unbekannter Relationship-Type '{rel_type}'!")
        print(f"     Erlaubt: {known_rels}")
    
    query = f"""
    MATCH (from {{id: $from_id}})
    MATCH (to {{id: $to_id}})
    CREATE (from)-[r:{rel_type}]->(to)
    RETURN r
    """
    
    return run_query(query, {'from_id': from_id, 'to_id': to_id})


def connect_locations(loc1_id, loc2_id):
    """
    Verbindet zwei Locations bidirektional mit ERREICHT
    
    Args:
        loc1_id: ID der ersten Location
        loc2_id: ID der zweiten Location
    """
    create_relationship(loc1_id, REL_ERREICHT, loc2_id)
    create_relationship(loc2_id, REL_ERREICHT, loc1_id)
    print(f"  ✓ Locations verbunden: {loc1_id} ↔ {loc2_id}")


def place_item(item_id, location_id):
    """
    Platziert ein Item an einer Location
    
    Args:
        item_id: ID des Items
        location_id: ID der Location
    """
    create_relationship(item_id, REL_IST_IN, location_id)
    print(f"  ✓ Item platziert: {item_id} → {location_id}")


def place_npc(npc_id, location_id):
    """
    Platziert einen NPC an einer Location
    
    Args:
        npc_id: ID des NPCs
        location_id: ID der Location
    """
    create_relationship(npc_id, REL_IST_IN, location_id)
    print(f"  ✓ NPC platziert: {npc_id} → {location_id}")


def place_player(player_id, location_id):
    """
    Platziert den Player an einer Location
    
    Args:
        player_id: ID des Players (normalerweise 'player')
        location_id: ID der Location
    """
    create_relationship(player_id, REL_IST_IN, location_id)
    print(f"  ✓ Player platziert: {player_id} → {location_id}")


def give_item_to_npc(item_id, npc_id):
    """
    Gibt einem NPC ein Item (ins Inventar)
    
    Args:
        item_id: ID des Items
        npc_id: ID des NPCs
    """
    create_relationship(npc_id, REL_TRÄGT, item_id)
    print(f"  ✓ Item ins NPC-Inventar: {npc_id} trägt {item_id}")


def make_key_unlock(key_id, target_id):
    """
    Schlüssel öffnet Tür/Truhe
    
    Args:
        key_id: ID des Schlüssels
        target_id: ID der Tür/Truhe
    """
    create_relationship(key_id, REL_ÖFFNET, target_id)
    print(f"  ✓ Schlüssel-Interaktion: {key_id} öffnet {target_id}")


def make_tool_light(tool_id, target_id):
    """
    Tool kann Target anzünden
    
    Args:
        tool_id: ID des Tools (z.B. Streichhölzer)
        target_id: ID des Targets (z.B. Fackel)
    """
    create_relationship(tool_id, REL_KANN_ANZÜNDEN, target_id)
    print(f"  ✓ Anzünd-Interaktion: {tool_id} zündet {target_id} an")


def make_tool_break(tool_id, target_id):
    """
    Tool kann Target zerbrechen
    
    Args:
        tool_id: ID des Tools (z.B. Hammer)
        target_id: ID des Targets (z.B. Truhe)
    """
    create_relationship(tool_id, REL_KANN_BRECHEN, target_id)
    print(f"  ✓ Zerbrechen-Interaktion: {tool_id} zerbricht {target_id}")


def make_light_illuminate(light_id, location_id):
    """
    Lichtquelle beleuchtet Location
    
    Args:
        light_id: ID der Lichtquelle (z.B. Fackel)
        location_id: ID der Location
    """
    create_relationship(light_id, REL_BELEUCHTET, location_id)
    print(f"  ✓ Beleuchtung: {light_id} beleuchtet {location_id}")


print("✓ Helper-Funktionen geladen")

✓ Helper-Funktionen geladen


In [25]:
# WARNUNG: Löscht ALLE Daten!
# Uncomment nur wenn du wirklich resetten willst:

def reset_db():
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")
        print("✓ Database reset - alle Nodes gelöscht")

reset_db()

✓ Database reset - alle Nodes gelöscht


In [26]:
emb_model = SentenceTransformer('deutsche-telekom/gbert-large-paraphrase-cosine')

## Constrains anlegen

In [27]:
# Constraints erstellen
print("Erstelle Constraints...")

constrain_queries = [
    'CREATE CONSTRAINT location_id IF NOT EXISTS FOR (l:Location) REQUIRE l.id IS UNIQUE',
    'CREATE CONSTRAINT item_id     IF NOT EXISTS FOR (i:Item)     REQUIRE i.id IS UNIQUE',
    'CREATE CONSTRAINT npc_id      IF NOT EXISTS FOR (n:NPC)      REQUIRE n.id IS UNIQUE',
    'CREATE CONSTRAINT player_id   IF NOT EXISTS FOR (p:Player)   REQUIRE p.id IS UNIQUE',
]

[run_query(query) for query in constrain_queries]
print("✓ Constraints erstellt")

# Property Indexes
print("\nErstelle Property Indexes...")
entity_indexes = [
    "CREATE INDEX location_name IF NOT EXISTS FOR (l:Location) ON (l.name)",
    "CREATE INDEX item_name     IF NOT EXISTS FOR (i:Item)     ON (i.name)",
    "CREATE INDEX npc_name      IF NOT EXISTS FOR (n:NPC)      ON (n.name)"
]

[run_query(query) for query in entity_indexes]
print("✓ Property Indexes erstellt")

# Vector Indexes
print("\nErstelle Vector Indexes...")
vector_indexes = [
    # Location Embeddings
    """
    CREATE VECTOR INDEX location_name_index IF NOT EXISTS
    FOR (l:Location) ON l.name_emb
    OPTIONS {indexConfig: {
      `vector.dimensions`: 768,
      `vector.similarity_function`: 'cosine'
    }}
    """,
    """
    CREATE VECTOR INDEX location_description_index IF NOT EXISTS
    FOR (l:Location) ON l.description_emb
    OPTIONS {indexConfig: {
      `vector.dimensions`: 768,
      `vector.similarity_function`: 'cosine'
    }}
    """,
    # Item Embeddings
    """
    CREATE VECTOR INDEX item_name_index IF NOT EXISTS
    FOR (i:Item) ON i.name_emb
    OPTIONS {indexConfig: {
      `vector.dimensions`: 768,
      `vector.similarity_function`: 'cosine'
    }}
    """,
    """
    CREATE VECTOR INDEX item_description_index IF NOT EXISTS
    FOR (i:Item) ON i.description_emb
    OPTIONS {indexConfig: {
      `vector.dimensions`: 768,
      `vector.similarity_function`: 'cosine'
    }}
    """,
    """
    CREATE VECTOR INDEX item_synonyms_index IF NOT EXISTS
    FOR (i:Item) ON i.synonyms_emb
    OPTIONS {indexConfig: {
      `vector.dimensions`: 768,
      `vector.similarity_function`: 'cosine'
    }}
    """,
    # NPC Embeddings
    """
    CREATE VECTOR INDEX npc_name_index IF NOT EXISTS
    FOR (n:NPC) ON n.name_emb
    OPTIONS {indexConfig: {
      `vector.dimensions`: 768,
      `vector.similarity_function`: 'cosine'
    }}
    """,
    """
    CREATE VECTOR INDEX npc_description_index IF NOT EXISTS
    FOR (n:NPC) ON n.description_emb
    OPTIONS {indexConfig: {
      `vector.dimensions`: 768,
      `vector.similarity_function`: 'cosine'
    }}
    """
]

for query in vector_indexes:
    run_query(query)

print("✓ Vector Indexes erstellt")

Erstelle Constraints...
✓ Constraints erstellt

Erstelle Property Indexes...
✓ Property Indexes erstellt

Erstelle Vector Indexes...
✓ Vector Indexes erstellt


## Spielwelt erstellen

Nutzt die Helper-Funktionen für typsichere Erstellung.

In [28]:
# Locations erstellen
print("Erstelle Locations...")

create_location(
    'taverne',
    "Mo's Taverne",
    'Eine alte Taverne, etwas heruntergekommen aber gemütlich. Hier wird jeden Abend gefeiert.'
)

create_location(
    'marktplatz',
    'Marktplatz',
    'Das ist der Marktplatz des kleinen, beschaulichen Ortes.'
)

create_location(
    'finsterwald',
    'Finsterwald',
    'Ein dunkler, übelriechender Wald voller Geister und Ängste.',
    is_dark=True,
    requires_light=True
)

create_location(
    'schmiede',
    'Alte Schmiede',
    'Eine verlassene Schmiede am Rande des Dorfes. Der Amboss steht noch, aber das Feuer ist längst erloschen.'
)

print("\n✓ Alle Locations erstellt")

Erstelle Locations...
  ✓ Location erstellt: taverne
  ✓ Location erstellt: marktplatz
  ✓ Location erstellt: finsterwald
  ✓ Location erstellt: schmiede

✓ Alle Locations erstellt


In [29]:
# Items erstellen
print("Erstelle Items...")

create_item(
    'truhe',
    'Alte Truhe',
    'Eine alte Truhe mit einem rostigen Schloss. Sie steht schon länger hier. Alt und verwittert.',
    synonyms=['Truhe', 'Kiste', 'Schatztruhe', 'Holztruhe'],
    is_container=True,
    is_locked=True,
    is_takeable=False
)

create_item(
    'schluessel',
    'Rostiger Schlüssel',
    'Ein rostiger Schlüssel der nur in ein ganz bestimmtes Schloss passt.',
    synonyms=['Schlüssel', 'Dietrich', 'Schloss-Öffner'],
    item_type='key',
    is_usable=True
)

create_item(
    'fackel',
    'Flackernde Fackel',
    'Eine alte Fackel. Noch nicht angezündet, aber bereit Licht zu spenden.',
    synonyms=['Fackel', 'Lichtquelle', 'Flamme'],
    is_lit=False,
    is_light_source=True,
    is_usable=True
)

create_item(
    'streichhoelzer',
    'Streichhölzer',
    'Eine kleine Schachtel mit Streichhölzern. Könnten nützlich sein.',
    synonyms=['Streichhölzer', 'Zündhölzer', 'Feuerzeug'],
    item_type='tool',
    is_usable=True
)

create_item(
    'hammer',
    'Schwerer Hammer',
    'Ein Schmiedehammer mit massivem Kopf. Schwer aber wirksam.',
    synonyms=['Hammer', 'Schmiedehammer', 'Werkzeug', 'schweres Ding'],
    item_type='tool',
    is_usable=True,
    weight=2.5
)

create_item(
    'beutel',
    'Lederbeutel',
    'Ein kleiner Lederbeutel. Es klappert etwas darin.',
    synonyms=['Beutel', 'Tasche', 'Ledertasche'],
    is_container=True
)

create_item(
    'schwert',
    'Altes Schwert',
    'Ein altes Schwert, das an der Wand der Taverne hängt. Rostig aber noch scharf.',
    synonyms=['Schwert', 'Klinge', 'Waffe'],
    item_type='weapon',
    is_usable=True,
    weight=1.5
)

create_item(
    'buch',
    'Vergilbtes Buch',
    'Ein altes Buch mit vergilbten Seiten. Scheint ein Rezeptbuch zu sein.',
    synonyms=['Buch', 'Rezeptbuch', 'Schrift'],
    is_readable=True
)

print("\n✓ Alle Items erstellt")

Erstelle Items...
  ✓ Item erstellt: truhe
  ✓ Item erstellt: schluessel
  ✓ Item erstellt: fackel
  ✓ Item erstellt: streichhoelzer
  ✓ Item erstellt: hammer
  ✓ Item erstellt: beutel
  ✓ Item erstellt: schwert
  ✓ Item erstellt: buch

✓ Alle Items erstellt


In [30]:
# NPCs und Player erstellen
print("Erstelle NPCs und Player...")

create_npc(
    'wirt',
    'Schenk',
    'Ein alter, grummiger Wirt, der seinen Gästen stets zu wenig einschenkt.',
    dialogue='Willkommen in meiner Taverne! Was willst du?',
    is_trader=True
)

create_npc(
    'haendler',
    'Wanderhändler',
    'Ein freundlicher Händler mit einem großen Rucksack voller Waren.',
    dialogue='Schau dir meine Waren an, Fremder!',
    is_trader=True
)

create_player(
    'player',
    'Player',
    'Hier könnte dein Name stehen!'
)

print("\n✓ NPCs und Player erstellt")

Erstelle NPCs und Player...
  ✓ NPC erstellt: wirt
  ✓ NPC erstellt: haendler
  ✓ Player erstellt: player

✓ NPCs und Player erstellt


## Welt verbinden

Locations, Items, NPCs und Player platzieren und verbinden.

In [31]:
# Player und NPCs platzieren
print("Platziere Player und NPCs...")

place_player('player', 'marktplatz')
place_npc('wirt', 'taverne')
place_npc('haendler', 'marktplatz')

print("\n✓ Player und NPCs platziert")

Platziere Player und NPCs...
  ✓ Player platziert: player → marktplatz
  ✓ NPC platziert: wirt → taverne
  ✓ NPC platziert: haendler → marktplatz

✓ Player und NPCs platziert


In [32]:
# Locations verbinden (Wegenetz)
print("Verbinde Locations...")

connect_locations('marktplatz', 'taverne')
connect_locations('marktplatz', 'finsterwald')
connect_locations('marktplatz', 'schmiede')

print("\n✓ Location-Netz erstellt")

Verbinde Locations...
  ✓ Locations verbunden: marktplatz ↔ taverne
  ✓ Locations verbunden: marktplatz ↔ finsterwald
  ✓ Locations verbunden: marktplatz ↔ schmiede

✓ Location-Netz erstellt


In [33]:
# Items platzieren
print("Platziere Items...")

place_item('schluessel', 'taverne')
place_item('truhe', 'finsterwald')
place_item('fackel', 'schmiede')
place_item('hammer', 'schmiede')
place_item('beutel', 'marktplatz')
place_item('schwert', 'taverne')
place_item('buch', 'taverne')

# Streichhölzer trägt der Wirt
give_item_to_npc('streichhoelzer', 'wirt')

print("\n✓ Items platziert")

Platziere Items...
  ✓ Item platziert: schluessel → taverne
  ✓ Item platziert: truhe → finsterwald
  ✓ Item platziert: fackel → schmiede
  ✓ Item platziert: hammer → schmiede
  ✓ Item platziert: beutel → marktplatz
  ✓ Item platziert: schwert → taverne
  ✓ Item platziert: buch → taverne
  ✓ Item ins NPC-Inventar: wirt trägt streichhoelzer

✓ Items platziert


In [34]:
# Item-Interaktionen (Quests)
print("Erstelle Item-Interaktionen...")

# Quest: Truhe öffnen (zwei Möglichkeiten)
make_key_unlock('schluessel', 'truhe')
make_tool_break('hammer', 'truhe')

# Quest: Fackel anzünden
make_tool_light('streichhoelzer', 'fackel')

# Quest: Finsterwald beleuchten
make_light_illuminate('fackel', 'finsterwald')

print("\n✓ Item-Interaktionen erstellt")

Erstelle Item-Interaktionen...
  ✓ Schlüssel-Interaktion: schluessel öffnet truhe
  ✓ Zerbrechen-Interaktion: hammer zerbricht truhe
  ✓ Anzünd-Interaktion: streichhoelzer zündet fackel an
  ✓ Beleuchtung: fackel beleuchtet finsterwald

✓ Item-Interaktionen erstellt


## Zusammenfassung

Überblick über die erstellte Spielwelt.

In [35]:
# Spielwelt-Statistik
print("="*60)
print(" "*20 + "SPIELWELT ERSTELLT")
print("="*60)

stats_query = """
MATCH (l:Location) WITH count(l) as locations
MATCH (i:Item) WITH locations, count(i) as items
MATCH (n:NPC) WITH locations, items, count(n) as npcs
MATCH (p:Player) WITH locations, items, npcs, count(p) as players
MATCH ()-[r]->() WITH locations, items, npcs, players, count(r) as relationships
RETURN locations, items, npcs, players, relationships
"""

stats = run_query(stats_query)[0]

print(f"\n📍 Locations:      {stats['locations']}")
print(f"📦 Items:          {stats['items']}")
print(f"👤 NPCs:           {stats['npcs']}")
print(f"🎮 Player:         {stats['players']}")
print(f"🔗 Relationships:  {stats['relationships']}")

print("\n" + "="*60)
print("Spielwelt bereit! Starte das Spiel mit: python src/main.py")
print("="*60)

                    SPIELWELT ERSTELLT

📍 Locations:      4
📦 Items:          8
👤 NPCs:           2
🎮 Player:         1
🔗 Relationships:  21

Spielwelt bereit! Starte das Spiel mit: python src/main.py


## Testwelt für Smart Parser

Minimale isolierte Testwelt für Parser-Tests (komplett getrennt von Haupt-Spielwelt).

In [36]:
# Test-Location
print("\nErstelle Test-Location...")

create_location(
    'testroom',
    'Testraum',
    'Ein Raum zum Testen der Parser-Funktionen.'
)

print("✓ Test-Location erstellt")


Erstelle Test-Location...
  ✓ Location erstellt: testroom
✓ Test-Location erstellt


In [37]:
# Test-Items mit Synonymen
print("Erstelle Test-Items...")

create_item(
    'test_schluessel',
    'Goldener Schlüssel',
    'Ein glänzender goldener Schlüssel',
    synonyms=['Schlüssel', 'goldener Schlüssel', 'Dietrich', 'Schloss-Öffner']
)

create_item(
    'test_schwert',
    'Rostiges Schwert',
    'Ein altes Schwert mit Rost',
    synonyms=['Schwert', 'rostiges Schwert', 'Klinge', 'Waffe', 'Eisenschwert']
)

create_item(
    'test_kristall',
    'Kristall',
    'Ein leuchtender Kristall',
    synonyms=['Kristall', 'Edelstein', 'Stein', 'funkelnder Brocken', 'glänzender Stein']
)

create_item(
    'test_laterne',
    'Laterne',
    'Eine alte flackernde Laterne',
    synonyms=['Laterne', 'Lampe', 'Leuchte', 'Licht']
)

create_item(
    'test_dolch',
    'Dolch',
    'Ein scharfer Dolch',
    synonyms=['Dolch', 'Messer', 'Klinge', 'Waffe']
)

create_item(
    'test_hammer',
    'Hammer',
    'Ein rostiger Schmiedehammer',
    synonyms=['Hammer', 'Werkzeug', 'Schmiedehammer', 'schweres Ding']
)

print("\n✓ Test-Items erstellt")

Erstelle Test-Items...
  ✓ Item erstellt: test_schluessel
  ✓ Item erstellt: test_schwert
  ✓ Item erstellt: test_kristall
  ✓ Item erstellt: test_laterne
  ✓ Item erstellt: test_dolch
  ✓ Item erstellt: test_hammer

✓ Test-Items erstellt


In [38]:
# Test-Player
print("Erstelle Test-Player...")

create_player(
    'testplayer',
    'Testplayer',
    'Ein Spieler zum Testen'
)

print("✓ Test-Player erstellt")

Erstelle Test-Player...
  ✓ Player erstellt: testplayer
✓ Test-Player erstellt


In [39]:
# Test-Welt verbinden
print("Verbinde Test-Welt...")

# Player platzieren
place_player('testplayer', 'testroom')

print("✓ Test-Player platziert")

Verbinde Test-Welt...
  ✓ Player platziert: testplayer → testroom
✓ Test-Player platziert


In [40]:
# Test-Items platzieren
print("Platziere Test-Items...")

place_item('test_schluessel', 'testroom')
place_item('test_schwert', 'testroom')
place_item('test_kristall', 'testroom')
place_item('test_laterne', 'testroom')
place_item('test_dolch', 'testroom')
place_item('test_hammer', 'testroom')

print("\n✓ Test-Items platziert")

# Test-Welt Statistik
print("\n" + "="*60)
print(" "*20 + "TESTWELT ERSTELLT")
print("="*60)

test_stats_query = """
MATCH (p:Player {id: 'testplayer'})-[:IST_IN]->(loc:Location)
MATCH (i:Item)-[:IST_IN]->(loc)
RETURN loc.name AS location, count(i) AS items
"""

test_stats = run_query(test_stats_query)[0]

print(f"\n📍 Test-Location:  {test_stats['location']}")
print(f"📦 Test-Items:     {test_stats['items']}")
print(f"🎮 Test-Player:    testplayer")

print("\n" + "="*60)
print("Testwelt bereit für Smart Parser Tests!")
print("="*60)

Platziere Test-Items...
  ✓ Item platziert: test_schluessel → testroom
  ✓ Item platziert: test_schwert → testroom
  ✓ Item platziert: test_kristall → testroom
  ✓ Item platziert: test_laterne → testroom
  ✓ Item platziert: test_dolch → testroom
  ✓ Item platziert: test_hammer → testroom

✓ Test-Items platziert

                    TESTWELT ERSTELLT

📍 Test-Location:  Testraum
📦 Test-Items:     6
🎮 Test-Player:    testplayer

Testwelt bereit für Smart Parser Tests!
